In [4]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
C.<x> = PolynomialRing(CC)
S = PowerSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m,x):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m,x):
    exn = exNo3c(n+2,m,x)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m,x):
    f = J(n,m,x)
    return f.truncate(n+1)
def Jcoefficient(n,m,x):
    f = J(n+2,m,x)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m,x): 
    f=J(n,m,x)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m,x):
    return xjNewStep(n-1,m,x)
def xjNewCoefficient(n,m,x):
    if n>0:return (xjNew(n,m,x)-xjNew(n-1,m,x))/x^n
    if n==0:return 1
def xjNewTest(n,m,x):
    return D(xjNewStep(n-1,m,x))
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def primeFactors(n):
    answer=[]
    lf=list(n.factor())
    length=len(lf)
    for k in [0..length-1]:
        pair=lf[k]
        (base,exponent)=pair
        answer=answer+[[base]]
    return(flatten(answer))
def primeFactorsIn(lst):
    ld=len(lst)
    primelist=[]
    for k in [0..ld-1]:
        a=ZZ(lst[k])
        primelist=primelist+[primeFactors(a)]
    return flatten(primelist)
def den(a):
    if a in ZZ:return 1
    else:return a.denominator()
def listdens(lst):
    return list(den(a) for a in lst)
def polynomialdenominator(poly):return lcm(listdens(poly))
def polynomialnumerator(poly):
    return expand(poly*polynomialdenominator(poly))
def num(a):return a.numerator()
def prime(n):
    P=Primes()
    return P.unrank(n-1)
def inclusiveListPrimesTo(n):
    if is_prime(n):
        return list(primes(n+1))
    else:
        return list(primes(n))
def chunk(k,list):
    chnk=[]
    for n in [k..len(list)-1]:
        chnk=chnk+[list[n]]
    return chnk
def drop(m,lst): return list(Set(lst).difference({m}))
def sort(list):
    answr=[]
    chnk=list
    for n in [0..len(list)-1]:
        m = min(chnk)
        answr=answr+[m]
        chnk=drop(m,chnk)
    return answr
def ord(p,n):return QQ.valuation(p)(n)
def base_conversion(number, base_from, base_to):  # by Vishnu Namboothiri
# https://ask.sagemath.org/question/9657/how-to-output-decimal-numbers-in-different-radix/
    if(base_from==10):
        return stripQuotationMarks((number.str(base=base_to)))
    else:
        decimal = Integer(str(number), base=base_from)
        return (stripQuotationMarks(decimal.str(base=base_to)))
def lastFactorWithIntegralFactors(polynomial):
    return polynomialnumerator(lastfactor(polynomial))
def numericalFactorWithIntegralFactors(polynomial):
    nf=numericalfactor(polynomial);
    pd=polynomialdenominator(lastfactor(polynomial))
    return nf/pd

def zeroLogsOverFieldNew(f,prime,power):
    answer=[]
    T = GF(prime^power)
    b=T.multiplicative_generator()
    
   
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
  
    a=0
    expr3=sage_eval(expr,locals={'x':a})
    if expr3==0:answer=answer+[-1]
        
    for i in [1..prime^power-1]:
        a=b^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            answer=answer+[i]
    return answer

def zeroLogsOverFieldDigitsNew(f,prime,power):
    answer=[]
    T = GF(prime^power)
    b=T.multiplicative_generator()
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
   
    a=0
    expr3=sage_eval(expr,locals={'x':a})
    if expr3==0:answer=answer+[-1]
    for i in [1..prime^power-1]:
        a=b^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            di=i.digits(prime)
            answer=answer+[di]
    return answer

def zerosOverFieldNew(f,prime,power):
    answer=[]
    T = GF(prime^power)
    b=T.multiplicative_generator()
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
    
    a=0
    expr3=sage_eval(expr,locals={'x':a})
    if expr3==0:answer=answer+[a]
        
    for i in [1..prime^power-1]:
        a=b^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            answer=answer+[a]
    return answer 

def fieldInverseNew(w,prime,power):
    T = GF(prime^power)
    b=T.multiplicative_generator()
    G.<t>= GF(prime^power)
    for i in [1..prime^power-1]:
        a=b^i
        if a*w == 1:return i
    return 0

def primeDivisors(n):
    pd=[]
    n=n
    fn=factor(n)
    ln=len(fn)
    for k in [0..ln-1]:
        part = fn[k]
        (prime,exponent) = part
        pd = pd + [prime]
    return pd 

def fOverField(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return ppoly


def factorOverField(f,prime,power):
    answer=[]
    T = GF(prime^power,'t')
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return fp



#######################################################
# I cannot reproduce this error message: "splitting tests below fail at  e.g. f = x + 1"
############################################################

def splitsTF(f,prime,power):
    T = GF(prime^power)
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx > 1:return(False)
    return(True)

def leastSplittingDegree(f,prime,bound):
    for power in [1..bound]:
        tf=splitsTF(f,prime,power)
        if tf == True:return power
    return -1

#########################################################

def firstNprimes(n):
    P=Primes()
    first=[]
    for k in [0..n-1]:
        first=first+[P.unrank(k)]
    return first

def slopeExistsTF(pairlist):
    lenth=len(pairlist)
    if lenth<3:return False
    slopes=[]
    for k in [1..lenth-1]:
        pair1=pairlist[k-1]
        abscissa1=pair1[0]
        ordinate1=pair1[1]
        pair2=pairlist[k]
        abscissa2=pair2[0]
        ordinate2=pair2[1]
        dx=abscissa2-abscissa1
        if dx == 0:return False
        dy=ordinate2-ordinate1
        slope=dy/dx
        slopes=slopes+[slope]
    slopes=Set(slopes)
    if len(slopes) == 1:
        return [True,slopes[0]]
    if len(slopes) > 1:
        return False     
    
def zeroOfLinearMonomial(monomial,x):return x-monomial

def logsAndMultiplicitiesOverField(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data

def zerosLogsAndMultiplicitiesOverField(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[0,-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[xm,i,exponent]]
    return data

def totalDegreeOverField(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    td=0
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        td=td+exponent
    return td

def zerosAndMultiplicitiesOverField(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            data=data+[[xm,exponent]]
    return data

In [ ]:
## printed output has been snipped

import pickle
wfile=open('/Users/barrybrent/run1jun21no1.txt','a')
data=[]
for m in [3..500]:
    print m
    data=data+[[m,normalJ(200,m)]]
s=pickle.dumps(str(data))
wfile.write(s)
wfile.close()
#########################
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
polydata=[]
import time
wfile = open('/Users/barrybrent/run3jun21no1.txt','a') # xJ polynomials
for n in [0..200]:
    start = time.time()
    data=[]
    for k in [0..len(s)-1]:
        print (n,k,time.time()-start)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n+2)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n,rl]] 
    finish = time.time()
    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()

In [4]:
import pickle
rfile = open('/Users/barrybrent/run3jun21no1.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print len(s)
print s[1]

201
[1, 3/8*x^4 + 1/2*x^2]


In [ ]:
logsAndMultiplicitiesOverField

In [11]:
def logsAndMultiplicitiesOverFieldTest(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    print gf
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data



In [17]:
import pickle
rfile = open('/Users/barrybrent/run3jun21no1.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
residue=0
bound=120
for k in [5..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        zerodata=[]
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        order=prime^sd
        td=totalDegreeOverField(poly,prime,sd)
        lm=logsAndMultiplicitiesOverFieldTest(poly,prime,sd)
        logs=[]
        digits=[]
        multiplicities=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            digit=log.digits(prime)
            logs=logs+[log]
            digits=digits+[digit]
            multiplicities=multiplicities+[lm[j][1]]
        
        print (index,sd,td,multiplicities)
        print logs; print digits
        print "---------------------------------------------------------------------------"

x^2 * (x + 1)^2 * (x + 2)^2 * (x + 3)^2 * (x + 4)^2
(5, 1, 10, [2, 2, 2, 2, 2])
[-1, 2, 3, 1, 4]
[[-1], [2], [3], [1], [4]]
---------------------------------------------------------------------------
(4) * x^2 * (x + 1)^4 * (x + 2)^4 * (x + 3)^4 * (x + 4)^4
(10, 1, 18, [2, 4, 4, 4, 4])
[-1, 2, 3, 1, 4]
[[-1], [2], [3], [1], [4]]
---------------------------------------------------------------------------
(2) * x^2 * (x + 1)^6 * (x + 2)^6 * (x + 3)^6 * (x + 4)^6
(15, 1, 26, [2, 6, 6, 6, 6])
[-1, 2, 3, 1, 4]
[[-1], [2], [3], [1], [4]]
---------------------------------------------------------------------------
(2) * x^2 * (x + 1)^8 * (x + 2)^8 * (x + 3)^8 * (x + 4)^8
(20, 1, 34, [2, 8, 8, 8, 8])
[-1, 2, 3, 1, 4]
[[-1], [2], [3], [1], [4]]
---------------------------------------------------------------------------
x^2 * (x + 1)^10 * (x + 2)^10 * (x + 3)^10 * (x + 4)^10
(25, 1, 42, [2, 10, 10, 10, 10])
[-1, 2, 3, 1, 4]
[[-1], [2], [3], [1], [4]]
----------------------------------------------

x^2 * (x + 1)^80 * (x + 2)^80 * (x + 3)^80 * (x + 4)^80
(200, 1, 322, [2, 80, 80, 80, 80])
[-1, 2, 3, 1, 4]
[[-1], [2], [3], [1], [4]]
---------------------------------------------------------------------------


In [18]:
import pickle
rfile = open('/Users/barrybrent/run3jun21no1.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
residue=1
bound=120
for k in [5..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        zerodata=[]
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        order=prime^sd
        td=totalDegreeOverField(poly,prime,sd)
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        logs=[]
        digits=[]
        multiplicities=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            digit=log.digits(prime)
            logs=logs+[log]
            digits=digits+[digit]
            multiplicities=multiplicities+[lm[j][1]]
        print "---------------------------------------------------------------------------"
        print (index,sd,td,multiplicities)
        print logs; print digits

---------------------------------------------------------------------------
(6, 1, 14, [2, 2, 2, 4, 4])
[-1, 3, 1, 2, 4]
[[-1], [3], [1], [2], [4]]
---------------------------------------------------------------------------
(11, 1, 22, [2, 4, 4, 6, 6])
[-1, 3, 1, 2, 4]
[[-1], [3], [1], [2], [4]]
---------------------------------------------------------------------------
(16, 1, 30, [2, 6, 6, 8, 8])
[-1, 3, 1, 2, 4]
[[-1], [3], [1], [2], [4]]
---------------------------------------------------------------------------
(21, 1, 38, [2, 8, 8, 10, 10])
[-1, 3, 1, 2, 4]
[[-1], [3], [1], [2], [4]]
---------------------------------------------------------------------------
(26, 1, 46, [2, 10, 10, 12, 12])
[-1, 3, 1, 2, 4]
[[-1], [3], [1], [2], [4]]
---------------------------------------------------------------------------
(31, 1, 54, [2, 12, 12, 14, 14])
[-1, 3, 1, 2, 4]
[[-1], [3], [1], [2], [4]]
---------------------------------------------------------------------------
(36, 1, 62, [2, 14, 1

In [19]:
import pickle
rfile = open('/Users/barrybrent/run3jun21no1.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
residue=2
bound=120
for k in [5..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        zerodata=[]
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        order=prime^sd
        td=totalDegreeOverField(poly,prime,sd)
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        logs=[]
        digits=[]
        multiplicities=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            digit=log.digits(prime)
            logs=logs+[log]
            digits=digits+[digit]
            multiplicities=multiplicities+[lm[j][1]]
        print "---------------------------------------------------------------------------"
        print (index,sd,td,multiplicities)
        print logs; print digits

---------------------------------------------------------------------------
(7, 4, 14, [1, 1, 1, 1, 2, 2, 2, 2, 2])
[143, 91, 455, 403, -1, 312, 468, 156, 624]
[[3, 3, 0, 1], [1, 3, 3], [0, 1, 3, 3], [3, 0, 1, 3], [-1], [2, 2, 2, 2], [3, 3, 3, 3], [1, 1, 1, 1], [4, 4, 4, 4]]
---------------------------------------------------------------------------
(12, 4, 22, [1, 1, 1, 1, 2, 4, 4, 4, 4])
[143, 91, 455, 403, -1, 312, 468, 156, 624]
[[3, 3, 0, 1], [1, 3, 3], [0, 1, 3, 3], [3, 0, 1, 3], [-1], [2, 2, 2, 2], [3, 3, 3, 3], [1, 1, 1, 1], [4, 4, 4, 4]]
---------------------------------------------------------------------------
(17, 4, 30, [1, 1, 1, 1, 2, 6, 6, 6, 6])
[143, 91, 455, 403, -1, 312, 468, 156, 624]
[[3, 3, 0, 1], [1, 3, 3], [0, 1, 3, 3], [3, 0, 1, 3], [-1], [2, 2, 2, 2], [3, 3, 3, 3], [1, 1, 1, 1], [4, 4, 4, 4]]
---------------------------------------------------------------------------
(22, 4, 38, [1, 1, 1, 1, 2, 8, 8, 8, 8])
[143, 91, 455, 403, -1, 312, 468, 156, 624]
[[3, 3, 0

---------------------------------------------------------------------------
(152, 4, 246, [1, 1, 1, 1, 2, 60, 60, 60, 60])
[143, 91, 455, 403, -1, 312, 468, 156, 624]
[[3, 3, 0, 1], [1, 3, 3], [0, 1, 3, 3], [3, 0, 1, 3], [-1], [2, 2, 2, 2], [3, 3, 3, 3], [1, 1, 1, 1], [4, 4, 4, 4]]
---------------------------------------------------------------------------
(157, 4, 254, [1, 1, 1, 1, 2, 62, 62, 62, 62])
[143, 91, 455, 403, -1, 312, 468, 156, 624]
[[3, 3, 0, 1], [1, 3, 3], [0, 1, 3, 3], [3, 0, 1, 3], [-1], [2, 2, 2, 2], [3, 3, 3, 3], [1, 1, 1, 1], [4, 4, 4, 4]]
---------------------------------------------------------------------------
(162, 4, 262, [1, 1, 1, 1, 2, 64, 64, 64, 64])
[143, 91, 455, 403, -1, 312, 468, 156, 624]
[[3, 3, 0, 1], [1, 3, 3], [0, 1, 3, 3], [3, 0, 1, 3], [-1], [2, 2, 2, 2], [3, 3, 3, 3], [1, 1, 1, 1], [4, 4, 4, 4]]
---------------------------------------------------------------------------
(167, 4, 270, [1, 1, 1, 1, 2, 66, 66, 66, 66])
[143, 91, 455, 403, -1, 312,

In [21]:
import pickle
rfile = open('/Users/barrybrent/run3jun21no1.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
residue=3
bound=120
for k in [5..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        zerodata=[]
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        order=prime^sd
        td=totalDegreeOverField(poly,prime,sd)
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        logs=[]
        digits=[]
        multiplicities=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            digit=log.digits(prime)
            logs=logs+[log]
            digits=digits+[digit]
            multiplicities=multiplicities+[lm[j][1]]
        print "---------------------------------------------------------------------------"
        print (index,sd,td,multiplicities)
        print logs; print digits

---------------------------------------------------------------------------
(8, 4, 16, [1, 1, 1, 1, 2, 2, 2, 3, 3])
[169, 481, 221, 533, -1, 312, 624, 468, 156]
[[4, 3, 1, 1], [1, 1, 4, 3], [1, 4, 3, 1], [3, 1, 1, 4], [-1], [2, 2, 2, 2], [4, 4, 4, 4], [3, 3, 3, 3], [1, 1, 1, 1]]
---------------------------------------------------------------------------
(13, 4, 24, [1, 1, 1, 1, 2, 4, 4, 5, 5])
[169, 481, 221, 533, -1, 312, 624, 468, 156]
[[4, 3, 1, 1], [1, 1, 4, 3], [1, 4, 3, 1], [3, 1, 1, 4], [-1], [2, 2, 2, 2], [4, 4, 4, 4], [3, 3, 3, 3], [1, 1, 1, 1]]
---------------------------------------------------------------------------
(18, 4, 32, [1, 1, 1, 1, 2, 6, 6, 7, 7])
[169, 481, 221, 533, -1, 312, 624, 468, 156]
[[4, 3, 1, 1], [1, 1, 4, 3], [1, 4, 3, 1], [3, 1, 1, 4], [-1], [2, 2, 2, 2], [4, 4, 4, 4], [3, 3, 3, 3], [1, 1, 1, 1]]
---------------------------------------------------------------------------
(23, 4, 40, [1, 1, 1, 1, 2, 8, 8, 9, 9])
[169, 481, 221, 533, -1, 312, 624, 468, 1

---------------------------------------------------------------------------
(153, 4, 248, [1, 1, 1, 1, 2, 60, 60, 61, 61])
[169, 481, 221, 533, -1, 312, 624, 468, 156]
[[4, 3, 1, 1], [1, 1, 4, 3], [1, 4, 3, 1], [3, 1, 1, 4], [-1], [2, 2, 2, 2], [4, 4, 4, 4], [3, 3, 3, 3], [1, 1, 1, 1]]
---------------------------------------------------------------------------
(158, 4, 256, [1, 1, 1, 1, 2, 62, 62, 63, 63])
[169, 481, 221, 533, -1, 312, 624, 468, 156]
[[4, 3, 1, 1], [1, 1, 4, 3], [1, 4, 3, 1], [3, 1, 1, 4], [-1], [2, 2, 2, 2], [4, 4, 4, 4], [3, 3, 3, 3], [1, 1, 1, 1]]
---------------------------------------------------------------------------
(163, 4, 264, [1, 1, 1, 1, 2, 64, 64, 65, 65])
[169, 481, 221, 533, -1, 312, 624, 468, 156]
[[4, 3, 1, 1], [1, 1, 4, 3], [1, 4, 3, 1], [3, 1, 1, 4], [-1], [2, 2, 2, 2], [4, 4, 4, 4], [3, 3, 3, 3], [1, 1, 1, 1]]
---------------------------------------------------------------------------
(168, 4, 272, [1, 1, 1, 1, 2, 66, 66, 67, 67])
[169, 481, 221, 

In [22]:
import pickle
rfile = open('/Users/barrybrent/run3jun21no1.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
residue=4
bound=120
for k in [5..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        zerodata=[]
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        order=prime^sd
        td=totalDegreeOverField(poly,prime,sd)
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        logs=[]
        digits=[]
        multiplicities=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            digit=log.digits(prime)
            logs=logs+[log]
            digits=digits+[digit]
            multiplicities=multiplicities+[lm[j][1]]
        print "---------------------------------------------------------------------------"
        print (index,sd,td,multiplicities)
        print logs; print digits

---------------------------------------------------------------------------
(9, 6, 18, [1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3])
[10773, 2961, 3717, 14805, 11529, 6993, -1, 7812, 15624, 11718, 3906]
[[3, 4, 0, 1, 2, 3], [1, 2, 3, 3, 4], [2, 3, 3, 4, 0, 1], [0, 1, 2, 3, 3, 4], [4, 0, 1, 2, 3, 3], [3, 3, 4, 0, 1, 2], [-1], [2, 2, 2, 2, 2, 2], [4, 4, 4, 4, 4, 4], [3, 3, 3, 3, 3, 3], [1, 1, 1, 1, 1, 1]]
---------------------------------------------------------------------------
(14, 6, 26, [1, 1, 1, 1, 1, 1, 2, 4, 4, 5, 5])
[10773, 2961, 3717, 14805, 11529, 6993, -1, 7812, 15624, 11718, 3906]
[[3, 4, 0, 1, 2, 3], [1, 2, 3, 3, 4], [2, 3, 3, 4, 0, 1], [0, 1, 2, 3, 3, 4], [4, 0, 1, 2, 3, 3], [3, 3, 4, 0, 1, 2], [-1], [2, 2, 2, 2, 2, 2], [4, 4, 4, 4, 4, 4], [3, 3, 3, 3, 3, 3], [1, 1, 1, 1, 1, 1]]
---------------------------------------------------------------------------
(19, 6, 34, [1, 1, 1, 1, 1, 1, 2, 6, 6, 7, 7])
[10773, 2961, 3717, 14805, 11529, 6993, -1, 7812, 15624, 11718, 3906]
[[3, 4, 0, 1, 

---------------------------------------------------------------------------
(114, 6, 186, [1, 1, 1, 1, 1, 1, 2, 44, 44, 45, 45])
[10773, 2961, 3717, 14805, 11529, 6993, -1, 7812, 15624, 11718, 3906]
[[3, 4, 0, 1, 2, 3], [1, 2, 3, 3, 4], [2, 3, 3, 4, 0, 1], [0, 1, 2, 3, 3, 4], [4, 0, 1, 2, 3, 3], [3, 3, 4, 0, 1, 2], [-1], [2, 2, 2, 2, 2, 2], [4, 4, 4, 4, 4, 4], [3, 3, 3, 3, 3, 3], [1, 1, 1, 1, 1, 1]]
---------------------------------------------------------------------------
(119, 6, 194, [1, 1, 1, 1, 1, 1, 2, 46, 46, 47, 47])
[10773, 2961, 3717, 14805, 11529, 6993, -1, 7812, 15624, 11718, 3906]
[[3, 4, 0, 1, 2, 3], [1, 2, 3, 3, 4], [2, 3, 3, 4, 0, 1], [0, 1, 2, 3, 3, 4], [4, 0, 1, 2, 3, 3], [3, 3, 4, 0, 1, 2], [-1], [2, 2, 2, 2, 2, 2], [4, 4, 4, 4, 4, 4], [3, 3, 3, 3, 3, 3], [1, 1, 1, 1, 1, 1]]
---------------------------------------------------------------------------
(124, 6, 202, [1, 1, 1, 1, 1, 1, 2, 48, 48, 49, 49])
[10773, 2961, 3717, 14805, 11529, 6993, -1, 7812, 15624, 11718, 3

In [16]:
def logsAndMultiplicitiesOverFieldTest(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    print gf
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data

import pickle
rfile = open('/Users/barrybrent/run3jun21no1.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
residue=6
bound=120
for k in [0..4]:
    index=s[k][0]
    zerodata=[]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    print poly
    sd=leastSplittingDegree(poly,prime,bound)
    order=prime^sd
    td=totalDegreeOverField(poly,prime,sd)
    lm=logsAndMultiplicitiesOverFieldTest(poly,prime,sd)
    logs=[]
    digits=[]
    multiplicities=[]
    for j in [0..len(lm)-1]:
        log=lm[j][0]
        digit=log.digits(prime)
        logs=logs+[log]
        digits=digits+[digit]
        multiplicities=multiplicities+[lm[j][1]]
    print (index,sd,td,multiplicities)
    print logs; print digits
    print "---------------------------------------------------------------------------"

x^2
x^2
(0, 1, 2, [2])
[-1]
[[-1]]
---------------------------------------------------------------------------
3*x^4 + 4*x^2
(3) * (x + t + 2) * (x + 4*t + 3) * x^2
(1, 2, 4, [1, 1, 2])
[3, 15, -1]
[[3], [0, 3], [-1]]
---------------------------------------------------------------------------
69*x^6 - 8*x^4 - 48*x^2
(4) * (x + 2*t^3 + 4*t + 1) * (x + 2*t^3 + t^2 + t + 3) * (x + 3*t^3 + t + 4) * (x + 3*t^3 + 4*t^2 + 4*t + 2) * x^2
(2, 4, 6, [1, 1, 1, 1, 2])
[143, 91, 455, 403, -1]
[[3, 3, 0, 1], [1, 3, 3], [0, 1, 3, 3], [3, 0, 1, 3], [-1]]
---------------------------------------------------------------------------
27*x^8 - 116*x^6 + 16*x^4 + 64*x^2
(2) * (x + 2) * (x + 3) * (x + t^2 + 2*t + 2) * (x + 4*t^2 + 3*t + 3) * (x + t^3 + 4*t^2 + 1) * (x + 4*t^3 + t^2 + 4) * x^2
(3, 4, 8, [1, 1, 1, 1, 1, 1, 2])
[468, 156, 169, 481, 221, 533, -1]
[[3, 3, 3, 3], [1, 1, 1, 1], [4, 3, 1, 1], [1, 1, 4, 3], [1, 4, 3, 1], [3, 1, 1, 4], [-1]]
-------------------------------------------------------------